In [1]:
import tkinter as tk
from tkinter import ttk
import requests
import cv2
import numpy as np
import imutils
from PIL import Image, ImageTk
import time

# Глобальные константы ⚙️
URL = "http://192.168.1.153:8080/shot.jpg"
IMG_WIDTH = 800
HSV_LOWER = np.array([40, 40, 40])
HSV_UPPER = np.array([80, 255, 255])
KERNEL = np.ones((3,3), np.uint8)
MIN_AREA = 300
MAX_TRACK_AGE = 2.0
MAX_DIST_SQ = 3600  # 60²

# Глобальное состояние 🧠
counts = {'Small': 0, 'Medium': 0, 'Large': 0}
tracks = {}
next_track_id = 0
app_running = False

# UI элементы (глобальные)
count_labels = {}
status_label = None
fps_label = None
tracks_label = None
video_label = None
btn_start = None

def reset_all():
    """🔄 Сброс счетчиков"""
    global counts, tracks, next_track_id
    counts = {'Small': 0, 'Medium': 0, 'Large': 0}
    tracks.clear()
    next_track_id = 0
    update_counters()

def process_frame(img):
    """🎯 Быстрая детекция + трекинг"""
    global tracks, next_track_id, counts
    
    # Маска HSV
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, HSV_LOWER, HSV_UPPER)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, KERNEL)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, KERNEL)
    
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Очистка старых треков
    current_age = time.time()
    tracks_to_del = [tid for tid, t in tracks.items() if current_age - t['time'] > MAX_TRACK_AGE]
    for tid in tracks_to_del:
        del tracks[tid]
    
    detections = []
    
    for contour in contours:
        if cv2.contourArea(contour) < MIN_AREA:
            continue
        
        x, y, w, h = cv2.boundingRect(contour)
        cx, cy = x + w//2, y + h//2
        size = np.sqrt(4 * cv2.contourArea(contour) / np.pi)
        
        # Категория и цвет
        if size < 50:
            cat, color = 'Small', (0, 255, 0)
        elif size < 100:
            cat, color = 'Medium', (0, 255, 255)
        else:
            cat, color = 'Large', (0, 0, 255)
        
        # 🔍 Поиск трека
        best_tid = None
        best_dist = float('inf')
        
        for tid, track in tracks.items():
            if track['cat'] != cat:
                continue
            dx = cx - track['cx']
            dy = cy - track['cy']
            dist_sq = dx*dx + dy*dy
            if dist_sq < MAX_DIST_SQ and dist_sq < best_dist:
                best_dist = dist_sq
                best_tid = tid
        
        track_text = ""
        if best_tid is not None:
            # Обновить трек
            tracks[best_tid] = {'cx': cx, 'cy': cy, 'size': size, 'cat': cat, 'time': current_age}
            track_text = f"ID:{best_tid}"
        else:
            # Новый огурец! ✅
            tid = next_track_id
            next_track_id += 1
            tracks[tid] = {'cx': cx, 'cy': cy, 'size': size, 'cat': cat, 'time': current_age}
            counts[cat] += 1
            track_text = f"NEW:{tid}"
        
        detections.append((x, y, w, h, cat, color, int(size), track_text))
    
    return detections

def update_counters():
    """Обновить счетчики на UI"""
    for cat in counts:
        count_labels[cat].config(text=str(counts[cat]))
    tracks_label.config(text=f"Tracks: {len(tracks)}")

def update_video():
    """🔄 Главный цикл обработки"""
    global app_running
    
    if not app_running:
        return
    
    try:
        # Загрузка кадра
        resp = requests.get(URL, timeout=1)
        nparr = np.frombuffer(resp.content, np.uint8)
        frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
        
        if frame is None:
            status_label.config(text="❌ Нет изображения")
            root.after(1000, update_video)
            return
        
        frame = imutils.resize(frame, width=IMG_WIDTH)
        
        # Детекция
        detections = process_frame(frame)
        
        # Отрисовка
        for x, y, w, h, cat, color, size, track_text in detections:
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            cv2.putText(frame, f"{cat} {track_text}", (x, y-10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        
        # Статистика
        total = sum(counts.values())
        cv2.putText(frame, f"TOTAL: {total}", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
        cv2.putText(frame, f"Tracks: {len(tracks)}", (10, 55),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
        
        # В Tkinter
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pil_img = Image.fromarray(rgb)
        tk_img = ImageTk.PhotoImage(pil_img)
        
        video_label.config(image=tk_img)
        video_label.image = tk_img  # удержать ссылку
        
        update_counters()
        status_label.config(text=f"✅ OK | {len(detections)} objs")
        
    except Exception as e:
        status_label.config(text=f"⚠️ {str(e)[:30]}")
    
    # Следующий кадр ~25 FPS
    root.after(30, update_video)

def toggle_detection():
    """▶️ Старт/пауза"""
    global app_running
    app_running = not app_running
    
    if app_running:
        btn_start.config(text="⏸️ PAUSE", bg="#ff8800")
        status_label.config(text="🚀 Запуск...")
        update_video()
    else:
        btn_start.config(text="▶️ START", bg="#00aa44")
        status_label.config(text="⏹️ Пауза")

# 🏗️ ПОСТРОЕНИЕ UI
root = tk.Tk()
root.title("🥒 ULTRA FAST Cucumber Counter v4.0 ⚡")
root.geometry("1100x750")
root.configure(bg='#0d1117')

# Header
header = tk.Label(root, text="🥒 Сортировка огурцов", 
                 font=('Arial', 22, 'bold'), fg="#00ff88", bg="#161b22")
header.pack(pady=15)

main_frame = tk.Frame(root, bg="#161b22")
main_frame.pack(fill='both', expand=True, padx=20, pady=10)

# Левый сайдбар
sidebar = tk.Frame(main_frame, bg="#21262d", width=300)
sidebar.pack(side='left', fill='y', padx=(0,15))
sidebar.pack_propagate(False)

tk.Label(sidebar, text="📷 Камера:", font=('Arial', 12, 'bold'), 
        fg='white', bg="#21262d").pack(pady=(20,5))
url_entry = tk.Entry(sidebar, font=('Arial', 11), width=32, justify='center')
url_entry.insert(0, URL)
url_entry.pack(pady=5)

# Кнопки
btn_frame = tk.Frame(sidebar, bg="#21262d")
btn_frame.pack(pady=25)

btn_start = tk.Button(btn_frame, text="▶️ START", command=toggle_detection,
                     font=('Arial', 16, 'bold'), bg="#00aa44", fg='white',
                     width=14, height=2, relief='flat')
btn_start.pack(pady=8)

tk.Button(btn_frame, text="🔄 RESET", command=reset_all,
         font=('Arial', 16, 'bold'), bg="#cc5500", fg='white',
         width=14, height=2, relief='flat').pack(pady=8)

# Счетчики
tk.Label(sidebar, text="📊 СЧЕТЧИКИ", font=('Arial', 16, 'bold'), 
        fg="#00ff88", bg="#21262d").pack(pady=(30,15))

size_info = [
    ('Small', '#00ff44', 0),
    ('Medium', '#ffaa00', 1),
    ('Large', '#ff4444', 2)
]

for name, color, _ in size_info:
    row = tk.Frame(sidebar, bg="#21262d")
    row.pack(fill='x', pady=8)
    tk.Label(row, text=f"{name}:", font=('Arial', 13, 'bold'), 
            fg=color, bg="#21262d").pack(side='left')
    lbl = tk.Label(row, text="0", font=('Arial', 28, 'bold'), 
                  fg=color, bg="#21262d")
    lbl.pack(side='right')
    count_labels[name] = lbl

# Статус
status_frame = tk.Frame(sidebar, bg="#21262d")
status_frame.pack(fill='x', pady=25)
status_label = tk.Label(status_frame, text="⏹️ Готов", 
                       font=('Arial', 13), fg="#888", bg="#21262d")
status_label.pack()
fps_label = tk.Label(status_frame, text="FPS: 0", 
                    font=('Arial', 13), fg="#888", bg="#21262d")
fps_label.pack()
tracks_label = tk.Label(status_frame, text="Tracks: 0", 
                       font=('Arial', 13), fg="#888", bg="#21262d")
tracks_label.pack()

# Видео панель
video_panel = tk.LabelFrame(main_frame, text="📹 LIVE VIDEO", 
                           font=('Arial', 15, 'bold'), fg='white', bg="#21262d")
video_panel.pack(side='right', fill='both', expand=True)

video_label = tk.Label(video_panel, bg='black', text="Нажмите START", 
                      font=('Arial', 18), fg='#aaa')
video_label.pack(expand=True, padx=15, pady=15)

# Финализация
root.protocol("WM_DELETE_WINDOW", root.quit)
root.mainloop()